In [ ]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from fixed_hedge_strategy import FixedHedgeStrategy
from dynamic_hedge_strategy import DynamicHedgeStrategy
from dynamic_hedge_atr_strategy import DynamicHedgeAtrStrategy
from datetime import datetime

In [ ]:
# 固定区间策略回测
params = {
    'hedge_range_param': 60,
    'hedge_multiple_param': 50
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2019, 1, 1),
    end=datetime(2020, 4, 1),
    rate=1/10000,
    slippage=0.002,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(FixedHedgeStrategy, params)

In [ ]:
# 动态区间策略回测
params = {
    'hedge_range_percent': 30,
    'hedge_multiple_percent': 100
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2019, 1, 1),
    end=datetime(2020, 4, 1),
    rate=1/10000,
    slippage=0.002,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(DynamicHedgeStrategy, params)

In [ ]:
# atr动态对冲回测
params = {
    'atr_range': 15,
    'hedge_multiple_percent': 80,
    'is_log': False
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2019, 1, 1),
    end=datetime(2020, 4, 1),
    rate=1/10000,
    slippage=0.002,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(DynamicHedgeAtrStrategy, params)

In [ ]:
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
# engine.show_chart()

In [ ]:
# 计算日线atr数据
import pandas as pd
from option_backtest import OptionBacktest

class CalAtr(OptionBacktest):
    def __init__(self):
        super().__init__()
        self.atr_window = 20
        self.atr_list = []
        self.atr_df = None
        
    def on_bar(self, bar):
        am = self.am
        am.update_bar(bar)
        if not am.inited:
            return
        atr = am.atr(self.atr_window)
        atr_dict = {'datetime': bar.date, 'atr': atr}
        self.atr_list.append(atr_dict)
        
    def stats_result(self):
        self.atr_df = pd.DataFrame(self.atr_list)
        self.atr_df.to_csv('510050_atr.csv', index=False)
        print("ATR数据保存成功")

atr_cal = CalAtr()
atr_cal.run_backtest()

In [ ]:
from datetime import datetime
df = pd.read_csv('510050_atr.csv', parse_dates=[1], index_col=1)
df['id'] = list(range(len(df)))
df.dtypes
d1 = datetime(2020, 4, 16)
last_day_id = int(df.loc[d1]['id'] - 1)
last_day_id
last_day = df.iloc[last_day_id]
last_day